In [2]:
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup
import requests
import re

## Slice full transcripts

In [3]:
directory = 'data/raw/transcripts_full'
transcripts_list = []
 
# iterate over files in that directory
for filename in os.listdir(directory):
    file = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(file) and directory + '/transcripts_' in file:
        print(file)
        
        with open(file, encoding='utf-8') as f:
            transcript_df = pd.read_csv(f)
            
            # replace NaN in Text field by empty string ''
            transcript_df['Text'] = transcript_df['Text'].fillna('')

            # replace automatic placeholder for empty cells by NaN
            transcript_df = transcript_df.replace('#PersonNumber#', np.nan)
            transcript_df = transcript_df.replace('#SpeakerFunction#', np.nan)
            transcript_df = transcript_df.replace('#CouncilId#', np.nan)
            transcript_df = transcript_df.replace('#Start#', np.nan)
            transcript_df = transcript_df.replace('#End#', np.nan)
            transcript_df = transcript_df.replace('#Function#', np.nan)
            transcript_df = transcript_df.replace('#LanguageOfText#', np.nan)
            
            # append df to list of df
            transcripts_list.append(transcript_df)
            
len(transcripts_list)

data/raw/transcripts_full/transcripts_48.csv
data/raw/transcripts_full/transcripts_49.csv
data/raw/transcripts_full/transcripts_47.csv
data/raw/transcripts_full/transcripts_46.csv
data/raw/transcripts_full/transcripts_50.csv
data/raw/transcripts_full/transcripts_51.csv


6

In [4]:
# count number of empty cells for each column

for transcript_df in transcripts_list:
    print('size: ', len(transcript_df))
    for col in transcript_df.columns:
        print(col, transcript_df[col].isnull().sum())
        
    print('-----')

size:  51852
ID 0
IdSubject 0
PersonNumber 10300
Text 0
MeetingCouncilAbbreviation 0
MeetingDate 0
IdSession 0
SpeakerFunction 10300
CouncilId 10300
Start 7406
End 7406
Function 10305
LanguageOfText 5919
-----
size:  53019
ID 0
IdSubject 0
PersonNumber 11054
Text 0
MeetingCouncilAbbreviation 0
MeetingDate 0
IdSession 0
SpeakerFunction 11054
CouncilId 11058
Start 7280
End 7280
Function 11056
LanguageOfText 12237
-----
size:  46051
ID 0
IdSubject 0
PersonNumber 5875
Text 0
MeetingCouncilAbbreviation 0
MeetingDate 0
IdSession 0
SpeakerFunction 5875
CouncilId 5908
Start 4305
End 4305
Function 5917
LanguageOfText 5417
-----
size:  39969
ID 0
IdSubject 0
PersonNumber 5025
Text 0
MeetingCouncilAbbreviation 0
MeetingDate 0
IdSession 0
SpeakerFunction 24941
CouncilId 5026
Start 3238
End 3238
Function 31133
LanguageOfText 5537
-----
size:  51716
ID 0
IdSubject 0
PersonNumber 11708
Text 0
MeetingCouncilAbbreviation 0
MeetingDate 0
IdSession 0
SpeakerFunction 11708
CouncilId 11713
Start 6778
End 6

In [5]:
# remove html tags from texts
count = 0

for transcript_df in transcripts_list:
    print('transcript idx: ', count)
    
    transcript_df['Text'] = transcript_df['Text'].apply(lambda text: BeautifulSoup(text, 'html.parser').text)
    count += 1

transcript idx:  0
transcript idx:  1
transcript idx:  2
transcript idx:  3
transcript idx:  4
transcript idx:  5


In [6]:
# save all transcripts by session

for transcript_df in transcripts_list:
    sessions_list = list(transcript_df['IdSession'].unique())
    for session_id in sessions_list:
        transcript_session = transcript_df.loc[transcript_df['IdSession'] == session_id]
        transcript_session.to_csv('data/raw/transcripts/transcript_' + str(session_id) + '.csv', encoding='utf-8')

## Pre-process transcript files

In [287]:
# open data files

with open('data/data_persons.csv', encoding='utf-8') as file:
    persons_df = pd.read_csv(file)

with open('data/data_businesses.csv', encoding='utf-8') as file:
    businesses_df = pd.read_csv(file)

with open('data/data_subjects.csv', encoding='utf-8') as file:
    subjects_df = pd.read_csv(file)
    
businesses_df

,ID,BusinessShortNumber,BusinessTypeName,Title,Description,InitialSituation,Proceedings,SubmittedText,SubmittedBy,BusinessStatusText,BusinessStatusDate,ResponsibleDepartmentAbbreviation,SubmissionDate,SubmissionSession
0,19780222,78.2220,Initiative parlementaire,Code pénal. Interruption de la grossesse (Gira...,Rapport de la Commission du 27.08.1979,NaN,NaN,NaN,Girard Gertrude,Liquidé,1981-03-10T00:00:00,NaN,1978-06-05T00:00:00,4311
1,19800226,80.2260,Initiative parlementaire,Constitutiion fédérale. Droit de l'entreprise ...,NaN,NaN,NaN,NaN,Jelmini Camillo,Liquidé,1983-03-03T00:00:00,Parl,1980-06-02T00:00:00,4311
2,19850019,85.0190,Objet du Conseil fédéral,Utilisation pacifique de l'énergie nucléaire. ...,Message et projet d'arrêté du 1er mai 1985 con...,<text><p>L'accord établit le cadre de droit in...,<text><p></p><p>Les deux conseils ont décidé s...,NaN,NaN,Liquidé,2003-12-16T00:00:00,DFAE,1985-05-01T00:00:00,4311
3,19850227,85.2270,Initiative parlementaire,Droit des assurances sociales,NaN,<text><p>Le droit suisse des assurances social...,<text><p>Le <b>Conseil des Etats</b> a accepté...,<text><p>A la suite de la motion visant une me...,Meier Josi J.,Liquidé,2000-10-06T00:00:00,Parl,1985-02-07T00:00:00,4311
4,19870069,87.0690,Objet du Conseil fédéral,Loi sur les chemins de fer. Modification,Message et projets de lois du 18 novembre 1987...,NaN,NaN,NaN,NaN,Liquidé,1987-11-18T00:00:00,DETEC,1987-11-18T00:00:00,4311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56869,20237263,23.7263,Heure des questions. Question,Assouplir les règles limitant la hauteur des i...,NaN,NaN,NaN,<text><p>La Suisse recèle un important potenti...,Gugger Niklaus-Samuel,Liquidé,2023-03-13T00:00:00,DETEC,2023-03-08T00:00:00,5118
56870,20237264,23.7264,Heure des questions. Question,Connaissances actuelles en matière de précipit...,NaN,NaN,NaN,<text><p>- De quelles informations le Conseil ...,Haab Martin,Liquidé,2023-03-13T00:00:00,DFI,2023-03-08T00:00:00,5118
56871,20237265,23.7265,Heure des questions. Question,Le langage inclusif est-il autorisé au Parleme...,NaN,NaN,NaN,"<text><p>En réponse à la question 23.7122, con...",Roduit Benjamin,Liquidé,2023-03-13T00:00:00,Parl,2023-03-08T00:00:00,5118
56872,20237266,23.7266,Heure des questions. Question,L'hydraulique comme réservoir d'énergie renouv...,NaN,NaN,NaN,<text><p>La Suisse doit tout mettre en oeuvre ...,Gmür Alois,Liquidé,2023-03-13T00:00:00,DETEC,2023-03-08T00:00:00,5118


In [257]:
persons_df

,PersonNumber,FirstName,LastName,GenderAsString,ParlGroupName,PartyName,PartyAbbreviation,DateJoining,DateLeaving,DateElection,DateOath,DateResignation,DateOfBirth,DateOfDeath
0,1,Pierre,Aguet,m,Groupe socialiste,Parti socialiste suisse,PSS,1995-12-04T00:00:00,1999-12-05T00:00:00,1995-12-04T00:00:00,1995-12-04T00:00:00,1999-12-05T00:00:00,1938-03-02T00:00:00,NaN
1,2,Heinz,Allenspach,m,Groupe libéral-radical,PLR.Les Libéraux-Radicaux,PLR,1979-11-26T00:00:00,1995-12-03T00:00:00,1979-11-26T00:00:00,1979-11-26T00:00:00,1995-12-03T00:00:00,1928-02-22T00:00:00,2022-09-16T00:00:00
2,6,Manfred,Aregger,m,Groupe libéral-radical,PLR.Les Libéraux-Radicaux,PLR,1995-12-04T00:00:00,1999-12-05T00:00:00,1995-12-04T00:00:00,1995-12-04T00:00:00,1999-12-05T00:00:00,1931-01-27T00:00:00,NaN
3,7,Geneviève,Aubry,f,Groupe libéral-radical,PLR.Les Libéraux-Radicaux,PLR,1979-11-26T00:00:00,1995-12-03T00:00:00,1979-11-26T00:00:00,1979-11-26T00:00:00,1995-12-03T00:00:00,1928-03-04T00:00:00,NaN
4,8,Rosmarie,Bär,f,Groupe des VERT-E-S,Liste libre (BE),LL,1987-11-30T00:00:00,1995-12-03T00:00:00,1987-11-30T00:00:00,1987-11-30T00:00:00,1995-12-03T00:00:00,1947-12-01T00:00:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3623,4329,Daniel,Ruch,m,Groupe libéral-radical,PLR.Les Libéraux-Radicaux,PLR,2022-06-13T00:00:00,NaN,2019-10-20T00:00:00,2022-06-13T00:00:00,NaN,1963-03-20T00:00:00,NaN
3624,4330,Alexandre,Berthoud,m,Groupe libéral-radical,PLR.Les Libéraux-Radicaux,PLR,2022-06-13T00:00:00,NaN,2019-10-20T00:00:00,2022-06-13T00:00:00,NaN,1977-06-29T00:00:00,NaN
3625,4331,Marc,Jost,m,Le Groupe du Centre. Le Centre. PEV.,Parti évangélique suisse,PEV,2022-11-28T00:00:00,NaN,2019-10-20T00:00:00,2022-11-28T00:00:00,NaN,1974-02-06T00:00:00,NaN
3626,4332,Mathilde,Crevoisier Crelier,f,Groupe socialiste,Parti socialiste suisse,PSS,2022-12-15T00:00:00,NaN,2019-10-20T00:00:00,2022-12-15T00:00:00,NaN,1980-01-05T00:00:00,NaN


In [275]:
# get all transcripts file names
directory = 'data/raw/transcripts'
transcripts_files_list = []
transcripts_list = []

# iterate over files in that directory
for filename in os.listdir(directory):
    file = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(file):
        transcripts_files_list.append(file)

# sort transcripts chronologically 
transcripts_files_list.sort()
len(transcripts_files_list)

117

In [276]:
# load all transcripts data
for file in transcripts_files_list:
    with open(file, encoding='utf-8') as f:
        transcript_df = pd.read_csv(f)
        # drop column 'Unnamed'
        transcript_df = transcript_df.drop(columns=['Unnamed: 0'])
        # append df to list of df
        transcripts_list.append(transcript_df)
            
len(transcripts_list)

117

In [277]:
for session_df in transcripts_list:
    session_id = session_df.loc[0, 'IdSession']
    print(session_id)
    
    # sort df
    session_df = session_df.sort_values(by=['MeetingCouncilAbbreviation', 'MeetingDate', 'Start', 'IdSubject'])
    
    # only keep rows with a PersonNumber
    session_df = session_df.dropna(subset=['PersonNumber'])
    
    #print(session_df['SpeakerFunction'].unique())
    #print(session_df['MeetingCouncilAbbreviation'].unique())
    #print(session_df['CouncilId'].unique())
    
    # drop rows when federal chancelor or council president is speaking
    functions_to_drop = ['BK-M', 'BK-F', 'P-M', 'P-F']
    session_df = session_df[~session_df['SpeakerFunction'].isin(functions_to_drop)]
    # drop rows when meeting is for both chambers (MeetingCouncilAbbreviation=V, for "Vereinigte Bundesversammlung")
    session_df = session_df[session_df['MeetingCouncilAbbreviation'] != 'V']
    
    # replace "[NB]" by a single space
    session_df.loc[:, 'Text'] = session_df.loc[:, 'Text'].replace(regex=r'\[NB\]', value=' ')
    # replace "[...]" by an empty string
    session_df.loc[:, 'Text'] = session_df.loc[:, 'Text'].replace(regex=r'\[.+?\]', value='')
    # set column PersonNumber type to int
    session_df = session_df.astype({"PersonNumber": int})
    
    # save df
    session_df.to_csv('data/transcripts/transcript_' + str(session_id) + '.csv', encoding='utf-8')

4601
4602
4603
4604
4605
4606
4607
4608
4609
4610
4611
4612
4613
4614
4615
4616
4617
4618
4619
4620
4701
4702
4703
4704
4705
4706
4707
4708
4709
4710
4711
4712
4713
4714
4715
4716
4717
4718
4801
4802
4803
4804
4805
4806
4807
4808
4809
4810
4811
4812
4813
4814
4815
4816
4817
4818
4819
4820
4901
4902
4903
4904
4905
4906
4907
4908
4909
4910
4911
4912
4913
4914
4915
4916
4917
4918
4919
4920
5001
5002
5003
5004
5005
5006
5007
5008
5009
5010
5011
5012
5013
5014
5015
5016
5017
5018
5019
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120


### Verification df processing

In [282]:
session_df = transcripts_list[4]
# sort df
session_df = session_df.sort_values(by=['MeetingCouncilAbbreviation', 'MeetingDate', 'Start', 'IdSubject'])
session_df

,ID,IdSubject,PersonNumber,Text,MeetingCouncilAbbreviation,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,End,Function,LanguageOfText
32,8071,1596,205.0,Ich begrüsse Sie sehr herzlich zur Wintersessi...,N,20001127,4605,NaN,1.0,2000-11-27T14:30:33,2000-11-27T14:44:36,P-M,DE
37,8076,1597,205.0,1. Wahl des Präsidenten des Nationalrates für ...,N,20001127,4605,NaN,1.0,2000-11-27T14:44:36,2000-11-27T14:45:28,NaN,NaN
36,8075,1597,205.0,Ergebnis der Wahl - Résultat du scrutin\nAusge...,N,20001127,4605,NaN,1.0,2000-11-27T14:55:57,2000-11-27T14:57:47,NaN,NaN
35,8074,1597,111.0,Für die ehrenvolle Wahl zum Nationalratspräsid...,N,20001127,4605,P-M,1.0,2000-11-27T14:57:47,2000-11-27T15:15:21,NaN,NaN
34,8073,1597,111.0,Ergebnis der Wahl - Résultat du scrutin\nAusge...,N,20001127,4605,P-M,1.0,2000-11-27T15:28:08,2000-11-27T15:30:11,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,10052,1940,111.0,"Herr Bundesrat Schmid, ich gratuliere Ihnen zu...",V,20001206,4605,P-M,1.0,2000-12-06T10:20:19,2000-12-06T10:22:52,NaN,DE
2012,10051,1940,111.0,3. Wahl des Bundespräsidenten für 2001 \n3. El...,V,20001206,4605,P-M,1.0,2000-12-06T10:35:58,2000-12-06T10:37:55,NaN,NaN
2011,10050,1940,111.0,4. Wahl des Vizepräsidenten des Bundesrates fü...,V,20001206,4605,P-M,1.0,2000-12-06T10:49:24,2000-12-06T10:50:57,NaN,NaN
2029,10068,1942,111.0,Ich erkläre die Sitzung der Vereinigten Bundes...,V,20001213,4605,P-M,1.0,2000-12-13T08:00:38,2000-12-13T08:01:49,NaN,DE


In [283]:
# only keep rows with a PersonNumber
session_df = session_df.dropna(subset=['PersonNumber'])
session_df

,ID,IdSubject,PersonNumber,Text,MeetingCouncilAbbreviation,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,End,Function,LanguageOfText
32,8071,1596,205.0,Ich begrüsse Sie sehr herzlich zur Wintersessi...,N,20001127,4605,NaN,1.0,2000-11-27T14:30:33,2000-11-27T14:44:36,P-M,DE
37,8076,1597,205.0,1. Wahl des Präsidenten des Nationalrates für ...,N,20001127,4605,NaN,1.0,2000-11-27T14:44:36,2000-11-27T14:45:28,NaN,NaN
36,8075,1597,205.0,Ergebnis der Wahl - Résultat du scrutin\nAusge...,N,20001127,4605,NaN,1.0,2000-11-27T14:55:57,2000-11-27T14:57:47,NaN,NaN
35,8074,1597,111.0,Für die ehrenvolle Wahl zum Nationalratspräsid...,N,20001127,4605,P-M,1.0,2000-11-27T14:57:47,2000-11-27T15:15:21,NaN,NaN
34,8073,1597,111.0,Ergebnis der Wahl - Résultat du scrutin\nAusge...,N,20001127,4605,P-M,1.0,2000-11-27T15:28:08,2000-11-27T15:30:11,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,10052,1940,111.0,"Herr Bundesrat Schmid, ich gratuliere Ihnen zu...",V,20001206,4605,P-M,1.0,2000-12-06T10:20:19,2000-12-06T10:22:52,NaN,DE
2012,10051,1940,111.0,3. Wahl des Bundespräsidenten für 2001 \n3. El...,V,20001206,4605,P-M,1.0,2000-12-06T10:35:58,2000-12-06T10:37:55,NaN,NaN
2011,10050,1940,111.0,4. Wahl des Vizepräsidenten des Bundesrates fü...,V,20001206,4605,P-M,1.0,2000-12-06T10:49:24,2000-12-06T10:50:57,NaN,NaN
2029,10068,1942,111.0,Ich erkläre die Sitzung der Vereinigten Bundes...,V,20001213,4605,P-M,1.0,2000-12-13T08:00:38,2000-12-13T08:01:49,NaN,DE


In [284]:
print(session_df['SpeakerFunction'].unique())
print(session_df['MeetingCouncilAbbreviation'].unique())
print(session_df['CouncilId'].unique())
session_df.loc[session_df['MeetingCouncilAbbreviation'] == 'V']

[nan 'P-M' 'BR-M' '1VP-F' 'BR-F' 'BK-F' '2VP-M' 'BPR-M' 'P-F' '1VP-M']
['N' 'S' 'V']
[ 1. 99. 98.  2. nan]


,ID,IdSubject,PersonNumber,Text,MeetingCouncilAbbreviation,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,End,Function,LanguageOfText
2028,10067,1941,111.0,Die Sitzung der Vereinigten Bundesversammlung ...,V,20001206,4605,P-M,1.0,2000-12-06T08:00:05,2000-12-06T08:01:30,NaN,DE
2016,10055,1940,111.0,1. Rücktritt von Herrn Bundesrat Adolf Ogi\n1....,V,20001206,4605,P-M,1.0,2000-12-06T08:01:30,2000-12-06T08:15:03,NaN,NaN
2017,10056,1940,428.0,"Herr Präsident, Ihre Worte haben mich gerührt....",V,20001206,4605,BPR-M,99.0,2000-12-06T08:15:03,2000-12-06T08:19:23,NaN,DE
2018,10057,1940,111.0,Ich danke Herrn Bundespräsident Ogi für seine ...,V,20001206,4605,P-M,1.0,2000-12-06T08:19:23,2000-12-06T08:20:53,NaN,DE
2019,10058,1940,84.0,Ich möchte im Namen der Fraktion der Schweizer...,V,20001206,4605,NaN,1.0,2000-12-06T08:20:53,2000-12-06T08:23:32,NaN,DE
2020,10059,1940,9.0,"Stellen Sie sich vor, die Zauberformel würde h...",V,20001206,4605,NaN,1.0,2000-12-06T08:23:32,2000-12-06T08:25:31,NaN,DE
2021,10060,1940,241.0,"Nous allons vivre, et le pays avec nous, aujou...",V,20001206,4605,NaN,1.0,2000-12-06T08:25:31,2000-12-06T08:30:54,NaN,FR
2022,10061,1940,70.0,"Le Parti libéral, vous le savez, n'est pas un ...",V,20001206,4605,NaN,1.0,2000-12-06T08:30:54,2000-12-06T08:34:37,NaN,FR
2023,10062,1940,111.0,Erster Wahlgang - Premier tour de scrutin\n[VS...,V,20001206,4605,P-M,1.0,2000-12-06T08:55:37,2000-12-06T08:57:44,NaN,NaN
2024,10063,1940,111.0,Zweiter Wahlgang - Deuxième tour de scrutin\n[...,V,20001206,4605,P-M,1.0,2000-12-06T09:12:05,2000-12-06T09:14:34,NaN,NaN


In [285]:
# drop rows when federal chancelor or council president is speaking
functions_to_drop = ['BK-M', 'BK-F', 'P-M', 'P-F']
session_df = session_df[~session_df['SpeakerFunction'].isin(functions_to_drop)]
# drop rows when meeting is for both chambers (MeetingCouncilAbbreviation=V, for "Vereinigte Bundesversammlung")
session_df = session_df[session_df['MeetingCouncilAbbreviation'] != 'V']
session_df

,ID,IdSubject,PersonNumber,Text,MeetingCouncilAbbreviation,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,End,Function,LanguageOfText
32,8071,1596,205.0,Ich begrüsse Sie sehr herzlich zur Wintersessi...,N,20001127,4605,NaN,1.0,2000-11-27T14:30:33,2000-11-27T14:44:36,P-M,DE
37,8076,1597,205.0,1. Wahl des Präsidenten des Nationalrates für ...,N,20001127,4605,NaN,1.0,2000-11-27T14:44:36,2000-11-27T14:45:28,NaN,NaN
36,8075,1597,205.0,Ergebnis der Wahl - Résultat du scrutin\nAusge...,N,20001127,4605,NaN,1.0,2000-11-27T14:55:57,2000-11-27T14:57:47,NaN,NaN
19,8058,1595,476.0,Zum ersten Mal seit 1991 unterbreitet der Bund...,N,20001127,4605,NaN,1.0,2000-11-27T16:25:08,2000-11-27T16:47:08,*,DE
17,8056,1595,492.0,Le budget 2001 de la Confédération se présente...,N,20001127,4605,NaN,1.0,2000-11-27T16:47:27,2000-11-27T17:12:14,*,FR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976,10015,1921,12.0,Sie haben wieder einen ausführlichen Bericht I...,S,20001214,4605,NaN,2.0,2000-12-14T09:30:31,2000-12-14T09:32:58,*,DE
1975,10014,1921,30.0,Comme notre présidente de la commission l'a di...,S,20001214,4605,NaN,2.0,2000-12-14T09:32:58,2000-12-14T09:37:33,NaN,FR
1974,10013,1921,375.0,Il est toujours délicat d'observer qu'une loi ...,S,20001214,4605,NaN,2.0,2000-12-14T09:37:33,2000-12-14T09:39:31,NaN,FR
1962,10001,1916,194.0,Sie haben den Bericht der Delegation bei der I...,S,20001214,4605,NaN,2.0,2000-12-14T09:40:58,2000-12-14T09:44:39,*,DE


In [286]:
# replace "[NB]" by a single space
session_df.loc[:, 'Text'] = session_df.loc[:, 'Text'].replace(regex=r'\[NB\]', value=' ')
# replace "[...]" by an empty string
session_df.loc[:, 'Text'] = session_df.loc[:, 'Text'].replace(regex=r'\[.+?\]', value='')
# set column PersonNumber type to int
session_df = session_df.astype({"PersonNumber": int})
session_df

,ID,IdSubject,PersonNumber,Text,MeetingCouncilAbbreviation,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,End,Function,LanguageOfText
32,8071,1596,205,Ich begrüsse Sie sehr herzlich zur Wintersessi...,N,20001127,4605,NaN,1.0,2000-11-27T14:30:33,2000-11-27T14:44:36,P-M,DE
37,8076,1597,205,1. Wahl des Präsidenten des Nationalrates für ...,N,20001127,4605,NaN,1.0,2000-11-27T14:44:36,2000-11-27T14:45:28,NaN,NaN
36,8075,1597,205,Ergebnis der Wahl - Résultat du scrutin\nAusge...,N,20001127,4605,NaN,1.0,2000-11-27T14:55:57,2000-11-27T14:57:47,NaN,NaN
19,8058,1595,476,Zum ersten Mal seit 1991 unterbreitet der Bund...,N,20001127,4605,NaN,1.0,2000-11-27T16:25:08,2000-11-27T16:47:08,*,DE
17,8056,1595,492,Le budget 2001 de la Confédération se présente...,N,20001127,4605,NaN,1.0,2000-11-27T16:47:27,2000-11-27T17:12:14,*,FR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976,10015,1921,12,Sie haben wieder einen ausführlichen Bericht I...,S,20001214,4605,NaN,2.0,2000-12-14T09:30:31,2000-12-14T09:32:58,*,DE
1975,10014,1921,30,Comme notre présidente de la commission l'a di...,S,20001214,4605,NaN,2.0,2000-12-14T09:32:58,2000-12-14T09:37:33,NaN,FR
1974,10013,1921,375,Il est toujours délicat d'observer qu'une loi ...,S,20001214,4605,NaN,2.0,2000-12-14T09:37:33,2000-12-14T09:39:31,NaN,FR
1962,10001,1916,194,Sie haben den Bericht der Delegation bei der I...,S,20001214,4605,NaN,2.0,2000-12-14T09:40:58,2000-12-14T09:44:39,*,DE


In [273]:
session_df.loc[session_df['ID'] == 71340]

,ID,IdSubject,PersonNumber,Text,MeetingCouncilAbbreviation,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,End,Function,LanguageOfText
66,71340,11355,326,"Das Büro hat die Wahl von Herrn Marc Suter, ge...",N,20070305,4716,Mit-M,1.0,2007-03-05T14:32:41,2007-03-05T14:34:33,B,DE
